In [2]:
import tensorflow
import keras
from keras.preprocessing import image
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.preprocessing.image import ImageDataGenerator #Used for augmenting the data
from keras.datasets import cifar10

from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np

import os
import argparse

Using TensorFlow backend.


In [ ]:
#opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
#opt = keras.optimizers.rmsprop(lr=0.000001, decay=1e-4)
#opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-7)

In [ ]:
def identity_block(input_tensor, filters, strides = (1,1)):
    
    filters1, filters2 = filters
    stride1, stride2 = strides 
         
    x = layers.Conv2D(filters1, (3, 3),
                      strides = stride1,
                      kernel_initializer='he_normal',
                      padding = 'same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)


    x = layers.Conv2D(filters2, (3,3),
                      strides = stride2,
                      kernel_initializer='he_normal',
                      padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.add([x, input_tensor])
    x = layers.Activation('relu')(x)
    return x

In [ ]:


def ResNets(depth=20, input_shape=(32,32,3)):
     
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 56 or 110 in [b])')    
    n = math.floor((depth-2)/6)
    print("value opf n is", n)
   
    
    if input_tensor is None:
        img_input = layers.Input(shape=input_shape)


    x = layers.Conv2D(64, (3, 3),
                      strides=(2, 2),
                      padding='same',
                      kernel_initializer='he_normal',
                      name='conv1')(x)
    x = layers.BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = layers.Activation('relu')(x)
    x = layers.ZeroPadding2D(padding=(1, 1), name='pool1_pad')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)


    
    for i in range(n):        
       x =  identity_block(x, (16,16), (1,1))(x)
    
    x =  identity_block(x, (32,32), (2,1))(x)
    
    for i in range(n-1):    
        x =  identity_block(x, (32,32), (1,1))(x)
        
    x =  identity_block(x, (64,64), (2,1))(x)   
    for i in range(n-1):    
        x =  identity_block(x, (64,64), (1,1))(x)
        
    x = layers.GlobalAveragePooling2D(name='block3_pool')(x)
    x = layers.Dense(10, activation='softmax', name='fc1')(x)
   

    model = Model(img_input, x, name='resnet34')
    
    return model


In [ ]:
acc = history_dict['acc']
val_acc = history_dict['val_acc']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

In [ ]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        
batch_losses = LossHistory()        

In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor='val_loss',
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-2,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=2,
        verbose=1),
    batch_losses
]

In [ ]:
#batch_losses = batch_losses.losses
#um_iterate = range(0, 390*6)
#num_epoch = [math.floor(x/390) for x in num_iterate]
#plt.plot(num_iterate, batch_losses, 'b', label='train_loss_plain_net20')

In [ ]:

def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(0.00001))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v2(input_shape, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:


def PlainNets(depth=20):
     
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 56 or 110 in [b])')    
    n = math.floor((depth-2)/6)
    print("value opf n is", n)
    model = keras.models.Sequential()
    model.add(Conv2D(64,(3,3), strides=2, padding='same',activation='relu', name='block1_conv1', input_shape=(32,32,3)))
    model.add(MaxPooling2D((3,3), strides=(2,2), name='block1_pool'))
    
    for i in range(2*n):
        conv_name_base = 'plain_' + str(depth) + 'block1' + '_branch' + str(i)
        model.add(Conv2D(16,(3,3), padding='same',activation='relu', name=conv_name_base))
    
    model.add(Conv2D(32, (3,3), padding='same', strides=2, activation='relu', name='block2_conv1'))  
    for i in range(2*n-1):
        conv_name_base = 'plain_' + str(depth) + 'block2' + '_branch' + str(i)
        model.add(Conv2D(32,(3,3), padding='same',activation='relu', name=conv_name_base))

    model.add(Conv2D(64, (3,3), padding='same', strides=2, activation='relu', name='block3_conv1'))
    for i in range(2*n-1):
        conv_name_base = 'plain_' + str(depth) + 'block3' + '_branch' + str(i)
        model.add(Conv2D(64,(3,3), padding='same',activation='relu', name=conv_name_base))
   
    model.add(GlobalAveragePooling2D(name='block3_pool')) 
    model.add(Dense(10, activation='softmax', name='fc1'))
    
    return model


In [ ]:


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(0.00001))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v2(input_shape, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [22]:
filepath = '/home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train'
count = 0
for subdir, dirs, files in os.walk(filepath):
    for folder in dirs:
        for folder_subdir, folder_dirs, folder_files in os.walk(os.path.join(subdir,folder)):
            print("folder_subdir :", folder_subdir)
            for file in folder_files:
            count=count + 1
                index = 0
                images.append(os.path.join(folder_subdir, file))
                labels.append(label_counter)
                if(count ==5):
                    print("folder_files :", folder_files)
print("count",count)    

folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n04596742
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03042490
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n01986214
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n01955084
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n01739381
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n01819313
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02699494
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03843555
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02098413
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02086646
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02099601
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03877472
folder_subdir : 

folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02906734
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n04418357
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n01930112
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03000247
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n04120489
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02111129
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n04019541
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n07730033
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n01796340
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03891251
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03014705
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n07714990
folder_subdir : 

folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n13044778
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02978881
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02009912
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n04597913
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03888257
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02095570
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03187595
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02111889
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n09246464
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n04590129
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n04270147
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n13052670
folder_subdir : 

folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03935335
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03887697
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02099849
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02977058
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03673027
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02607072
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02823428
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n07697537
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n07760859
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n04252225
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n04208210
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n01729977
folder_subdir : 

folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02012849
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02112350
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02319095
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n04070727
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02488291
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n01675722
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02280649
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02666196
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03980874
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02132136
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03425413
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03770679
folder_subdir : 

folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n01689811
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02219486
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02120505
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n04296562
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02086240
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02268853
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03733131
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n04380533
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02256656
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03983396
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n01695060
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n04483307
folder_subdir : 

folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03598930
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03272010
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02487347
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02093991
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03895866
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n01698640
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03691459
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03764736
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n01768244
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03661043
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03666591
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03788365
folder_subdir : 

folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n01770081
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n01877812
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02095889
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n04033995
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02389026
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n01944390
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02105855
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n02091635
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n07614500
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n03947888
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n01773549
folder_subdir : /home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/train/n04127249
folder_subdir : 

In [56]:
def get_val_data(filepath):
    label_file = open('/home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/ILSVRC2012_validation_ground_truth.txt',"r")
    labels_list = label_file.read().splitlines()
    print("print this",labels_list[5])
    label_counter = 0
    images = []
    labels = []
    for subdir, dirs, files in os.walk(filepath):
        for file in files:
            index = 0
            images.append(os.path.join(subdir, file))
            labels.append(labels_list[label_counter])
            label_counter = label_counter + 1    
    return images, labels

val_path = '/home/xena/Desktop/manoranjan/ILSVRC/Data/CLS-LOC/val' 
val_images, val_labels = get_val_data(val_path)

print this 482


In [58]:
val_labels[5]

'482'

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
print("x_train shape :",x_train.shape)
batch_size=32
num_classes=10
epochs=100
data_augmentation = True
num_pred=20
save_dir = os.path.join(os.getcwd(), 'saved_model')

x_train shape : (50000, 32, 32, 3)


In [4]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
#Model

model = keras.models.Sequential()
model.add(Conv2D(32,(3,3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Dropout(0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


In [6]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [7]:
model.compile(loss='categorical_crossentropy',
             optimizer=opt,
             metrics=['accuracy'])
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train= x_train/255
x_test = x_test/255

In [8]:
model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data=(x_test, y_test), shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 11s 213us/step - loss: 1.7068 - acc: 0.3785 - val_loss: 1.4486 - val_acc: 0.4775
Epoch 2/100
50000/50000 [==============================] - 9s 177us/step - loss: 1.4062 - acc: 0.4957 - val_loss: 1.2873 - val_acc: 0.5399
Epoch 3/100
50000/50000 [==============================] - 9s 178us/step - loss: 1.2701 - acc: 0.5493 - val_loss: 1.1530 - val_acc: 0.5944
Epoch 4/100
50000/50000 [==============================] - 9s 178us/step - loss: 1.1672 - acc: 0.5911 - val_loss: 1.1251 - val_acc: 0.6008
Epoch 5/100
50000/50000 [==============================] - 9s 179us/step - loss: 1.0905 - acc: 0.6175 - val_loss: 1.0450 - val_acc: 0.6349
Epoch 6/100
50000/50000 [==============================] - 9s 179us/step - loss: 1.0293 - acc: 0.6420 - val_loss: 0.9672 - val_acc: 0.6656
Epoch 7/100
50000/50000 [==============================] - 9s 180us/step - loss: 0.9780 - acc: 0.6571 - val_loss: 0

In [ ]:
def convert_to_numpy_array(dataset):
    data_length = len(train_images)
    index = 0
    data = [[[[]]]]   #np.zeros((data_length, 256,256,3))
    Label = []#np.zeros((data_length))
    while index < data_length :
        try:
            img = load_img(dataset[index])
            data[index] = img_to_array(img)
            data[index] = data[index]/256
            
            Label[index] = dataset[index]
            index = index + 1
        except:
            print("This image couldnot be processed {}".format(dataset[index]))
            index = index + 1
    return data, keras.utils.to_categorical(Label, num_classes)   

In [ ]:
def PlainNet_20():
    
    model = keras.models.Sequential()
    model.add(Conv2D(64,(3,3), padding='same', strides=2, activation='relu', name='block1_conv1', input_shape=(32,32,3)))
    #model.add(MaxPooling2D((3,3), strides=(2,2), name='block1_pool'))
    
    model.add(Conv2D(16,(3,3), padding = 'same', activation='relu', name='block1_conv2'))
    model.add(Conv2D(16,(3,3), padding = 'same', activation='relu', name='block1_conv3'))
    model.add(Conv2D(16,(3,3), padding = 'same', activation='relu', name='block1_conv4'))
    model.add(Conv2D(16,(3,3), padding = 'same', activation='relu', name='block1_conv5'))
    model.add(Conv2D(16,(3,3), padding = 'same', activation='relu', name='block1_conv6'))
    model.add(Conv2D(16,(3,3), padding = 'same', activation='relu', name='block1_conv7'))
    
    model.add(Conv2D(32, (3,3), padding='same', strides=2, activation='relu', name='block2_conv1'))
    model.add(Conv2D(32, (3,3), padding='same', activation='relu', name='block2_conv2'))
    model.add(Conv2D(32, (3,3), padding='same', activation='relu', name='block2_conv3'))
    model.add(Conv2D(32, (3,3), padding='same', activation='relu', name='block2_conv4'))
    model.add(Conv2D(32, (3,3), padding='same', activation='relu', name='block2_conv5'))
    model.add(Conv2D(32, (3,3), padding='same', activation='relu', name='block2_conv6'))

    
    model.add(Conv2D(64, (3,3), padding='same', strides=2, activation='relu', name='block3_conv1'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu', name='block3_conv2'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu', name='block3_conv3'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu', name='block3_conv4'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu', name='block3_conv5'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu', name='block3_conv6'))
   
    model.add(GlobalAveragePooling2D(name='block3_pool')) 
    #model.add(Flatten(name = 'flatten'))
    model.add(Dense(10, activation='softmax', name='fc1'))
    return model

In [ ]:
#train_data, train_labels = convert_to_numpy_array(train_images)